<a href="https://colab.research.google.com/github/r42arty/hse/blob/main/mod4/DL/DL_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основы машинного обучения, ИИМУП

## НИУ ВШЭ, 2024-25 учебный год

# Домашнее задание 3: Рекуррентные нейронные сети

Задание выполнил(а):

    Рубцов Артемий

## Общая информация

__Внимание!__  


* Домашнее задание выполняется самостоятельно
* Не допускается помощь в решении домашнего задания от однокурсников или третьих лиц. «Похожие» решения считаются плагиатом, и все задействованные студенты — в том числе и те, у кого списали, — не могут получить за него больше 0 баллов
* Использование в решении домашнего задания генеративных моделей (ChatGPT и так далее) за рамками справочной и образовательной информации для генерации кода задания — считается плагиатом, и такое домашнее задание оценивается в 0 баллов
* Старайтесь сделать код как можно более оптимальным. Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from collections import Counter, defaultdict
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np


In [12]:
BIOTECH_NEWS = 'https://www.dropbox.com/scl/fi/v624fdeg3bikyxyvr93i3/biotech_news.tsv?rlkey=iaekqfwtbqswl8vd5dpnd9d92&st=mzgbw5km&dl=1'

In [13]:
dataset = pd.read_csv(BIOTECH_NEWS, sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


## Предобработка лейблов


__Задание 1 (1.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [14]:
dataset['labels'] = dataset['labels'].apply(lambda x: x.split(','))

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(dataset['labels'])

## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [18]:
texts_train, texts_test, y_train, y_test = train_test_split(
    dataset['text'],
    y,
    test_size=0.2,  # do not change this
    random_state=0,  # do not change this
)

y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

__Задание 2 (1.5 балла)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [19]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))

all_words = []
for txt in texts_train:
    all_words += txt.split()

counts = Counter(all_words)

rare = set([w for w in counts if counts[w] < 5])
common = set([w for w, _ in counts.most_common(50)])

def clean(text):
    return ' '.join([w for w in text.split() if w not in stops and w not in rare and w not in common])

texts_train_clean = texts_train.apply(clean)
texts_test_clean = texts_test.apply(clean)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


__Задание 3 (2 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +5 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [20]:
corpus = []
for text in texts_train_clean:
    for word in text.strip().split():
        chars = list(word) + ['</w>']
        corpus.append(chars)

word_freq = Counter([' '.join(w) for w in corpus])

def get_stats(freqs):
    pairs = defaultdict(int)
    for word, count in freqs.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += count
    return pairs

def merge_pair(pair, freqs):
    out = {}
    old = ' '.join(pair)
    new = ''.join(pair)
    for word in freqs:
        out[word.replace(old, new)] = freqs[word]
    return out

for _ in range(1000):
    pairs = get_stats(word_freq)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    word_freq = merge_pair(best, word_freq)

vocab = set()
for word in word_freq:
    vocab.update(word.split())

token_to_idx = {tok: i + 1 for i, tok in enumerate(sorted(vocab))}

def bpe_tokenize(text, token_to_idx):
    tokens = []
    for word in text.strip().split():
        chars = list(word) + ['</w>']
        while True:
            pairs = [(chars[i], chars[i + 1]) for i in range(len(chars) - 1)]
            found = False
            for i in range(len(pairs)):
                pair = pairs[i]
                if ''.join(pair) in token_to_idx:
                    chars = chars[:i] + [''.join(pair)] + chars[i+2:]
                    found = True
                    break
            if not found:
                break
        tokens += chars
    return [token_to_idx.get(t, 0) for t in tokens]

X_train_bpe = [bpe_tokenize(t, token_to_idx) for t in texts_train_clean]
X_test_bpe = [bpe_tokenize(t, token_to_idx) for t in texts_test_clean]

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [21]:
def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс.

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [ ]:
class BpeDataset(Dataset):
    def __init__(self, token_seqs, labels):
        self.seqs = token_seqs
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        return torch.tensor(self.seqs[idx], dtype=torch.long), self.labels[idx]

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_batch = [x[:512] for x in x_batch]
    x_batch = pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_batch = torch.stack(y_batch)
    return x_batch, y_batch

train_dataset = BpeDataset(X_train_bpe, y_train)
test_dataset = BpeDataset(X_test_bpe, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# RNN, боль моя ...... попытка запуска №37
class ImprovedRnnClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, hidden_size=256, output_dim=55):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, emb_dim, padding_idx=0)
        self.rnn = nn.RNN(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=2,
            dropout=0.3,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        out, _ = self.rnn(embedded)
        mask = (x != 0).unsqueeze(-1).float()
        out = out * mask
        pooled = out.sum(1) / (mask.sum(1) + 1e-6)
        return self.fc(self.dropout(pooled))

# инициализируем
vocab_size = len(token_to_idx)
model = ImprovedRnnClassifier(vocab_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# лосс с учётом дисбаланса
label_freq = torch.tensor(y_train).sum(dim=0)
pos_weight = (len(y_train) - label_freq) / (label_freq + 1e-6)
pos_weight = pos_weight.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# трэйнинг
for epoch in range(epoch_count):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'epoch {epoch+1} loss: {total_loss:.4f}')

# топ К предсказания (топ-3 предательств в аниме)
def get_topk_predictions(y_probs, k=3):
    y_pred = np.zeros_like(y_probs)
    topk = np.argsort(y_probs, axis=1)[:, -k:]
    for i in range(len(y_probs)):
        y_pred[i, topk[i]] = 1
    return y_pred

# оценко
model.eval()
all_probs, all_targets = [], []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        logits = model(x_batch)
        probs = torch.sigmoid(logits).cpu()
        all_probs.append(probs)
        all_targets.append(y_batch)

y_probs = torch.cat(all_probs).numpy()
y_true = torch.cat(all_targets).numpy()

y_pred = get_topk_predictions(y_probs, k=3)
f1 = f1_score(y_true, y_pred, average='macro')
 # Ф-1.... Формула-1
print(f"\nF-1: {f1:.2f}")

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [ ]:
# your code here

__Задание 6 (1 балл).__ В этом задании у вас есть две опции на выбор: добавить __двунаправленность__ для LSTM _или_ добавить __многослойность__. Можно сделать и то, и другое, но дополнительных баллов за это мы не дадим, только бесконечный респект. Обе модификации реализуются довольно просто (буквально 4 строчки кода, если вы аккуратно реализовали модель) и дают примерно одинаковый прирост в качестве. Сделайте выводы: стоит ли увеличивать размер модели в несколько раз?

In [ ]:
# your code here